In [1]:
import tushare as ts
import seaborn as sns
import keras
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
from keras.models import Model
from keras import backend as K
from keras.layers import Input, Dense, LSTM, Dropout, Bidirectional, BatchNormalization, Activation
from keras.callbacks import EarlyStopping, TensorBoard
from keras.optimizers import RMSprop, Adam
from keras.initializers import Orthogonal
from tcn import TCN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (
    roc_auc_score, roc_curve, auc, f1_score,
    confusion_matrix, recall_score, 
    precision_recall_fscore_support)
set_matplotlib_formats('retina')
pd.set_option('display.max_columns', 100)

Using TensorFlow backend.


In [2]:
train = pd.read_csv('train_info.csv')
label = train['46']
train.drop('46', axis = 1, inplace =True)
#train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
0,0.040713,0.040713,0.044665,0.047307,0.062763,3.075524e-07,0.000050,0.014867,0.039876,0.986541,0.877902,0.103702,0.005488,0.165285,0.048398,0.000856,0.000856,0.011603,0.000386,0.011603,0.000386,0.580032,0.489082,0.153621,0.877379,0.004028,0.424978,1.0,0.453911,0.427022,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
1,0.020103,0.020103,0.017720,0.016070,0.020665,6.260488e-07,0.000028,0.027492,0.039855,0.899863,0.877184,0.271962,0.005485,0.164983,0.045906,0.000675,0.000675,0.011423,0.000384,0.011423,0.000384,0.579816,0.488942,0.153596,0.877478,0.005678,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.042216,0.042216,0.023471,0.023994,0.039308,7.423608e-07,0.000005,0.010256,0.039861,0.907961,0.876419,0.178934,0.005489,0.165431,0.044970,0.000905,0.000905,0.011651,0.000382,0.011651,0.000382,0.580096,0.488957,0.153634,0.877350,0.000474,0.292186,1.0,0.241400,0.293591,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
3,0.017561,0.017561,0.012516,0.010812,0.017399,5.145216e-07,0.000017,0.020576,0.039854,0.872822,0.877618,0.447431,0.005498,0.166091,0.050085,0.001916,0.001916,0.012651,0.000534,0.012651,0.000534,0.580587,0.489344,0.153641,0.877343,0.019254,0.249332,1.0,0.345085,0.250531,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.074037,0.074037,0.018363,0.014760,0.020305,4.278857e-07,0.000048,0.026193,0.039859,0.885592,0.876835,0.160976,0.005503,0.166678,0.049243,0.002698,0.002698,0.013425,0.000414,0.013425,0.000414,0.580905,0.489014,0.153706,0.877373,0.003747,0.455142,1.0,0.380463,0.457331,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [3]:
test = pd.read_csv('test_info.csv')
target = test['46']
test.drop('46', axis = 1, inplace =True)
#test.head()

In [4]:
train_price = pd.read_csv('train_price.csv')
train = pd.concat([train_price, train], axis = 1)
test_price = pd.read_csv('test_price.csv')
test = pd.concat([test_price, test], axis = 1)

In [16]:
import lightgbm as lgb

In [21]:
params = {'task':'train',
          'boosting_type':'gbdt',
          'num_leaves': 31,
          'learning_rate': 0.15, 
          'bagging_freq': 2, 
          'max_bin':256,
          'num_threads': 32,
          'objective': 'binary', 
         }
lgb_train = lgb.Dataset(train, label)
clf = lgb.train(params,lgb_train,num_boost_round=3000)

In [56]:
pred = clf.predict(test)

In [57]:
roc_auc_score(target, pred)

0.9100981186790188

In [71]:
y_class = [1 if p > 0.25 else 0 for p in pred]
confusion_matrix(target, y_class)

array([[44534,  5507],
       [ 3796, 12414]], dtype=int64)

In [72]:
from sklearn.metrics import f1_score
f1_score(target, y_class)

0.7274325393337435

In [55]:
clf.save_model('lgb_model.h5')